In [8]:
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd

In [9]:
import sys

sys.executable

'C:\\ProgramData\\Anaconda3\\python.exe'

In [10]:
driver_path = './chromedriver.exe'

In [11]:
def biz_khan(query,driver_path,page):
    domain = '세계일보'
    driver = webdriver.Chrome(executable_path=driver_path)
        
    current_page = 1
    last_page =(int(page)-1)*10+1
    
    head_list = []
    data_list = []
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path=driver_path)
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1022&nso=&is_sug_officeid=0'
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page)+ news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()


        for url in url_list:
            driver = webdriver.Chrome(executable_path=driver_path)  # for Windows
            try:
                driver.get(url)
                time.sleep(1)
            except:
                print('link error')
                continue
            # try문 안에서 각자 언론사에 맞게 수정
            try:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(soup.select('#title_sns')[0].text)
                data = soup.find('article',class_='viewBox').text
                data_list.append(''.join(data))
            except:
                print(url)
                print('None_type')
            driver.close()
        current_page+=10
        
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

biz = biz_khan('풀무원 바른먹거리',driver_path,5)
biz

link error
http://www.economysegye.com/articles/view.html?aid=20130326001515&cid=7111000000000&OutUrl=naver
None_type
link error


,언론사,제목,내용
0,세계일보,"풀무원다논, 플립형 토핑 요거트 ‘요거톡’ 누적 판매량 1000만개 달성",\n요거트 전문 기업 풀무원다논(대표 정희련)은 플립형 토핑 요거트 ‘요거톡’이 누...
1,세계일보,"풀무원푸드머스, 롯데하이마트와 어린이집 전문 방역 서비스 제공 MO",\n영유아 급식시장을 선도하고 있는 풀무원푸드머스가 롯데하이마트와 손잡고 키즈(Ki...
2,세계일보,"풀무원, 어린이 바른먹거리 교육에 ‘코로나19 예방 교육’ 실시",\n신종코로나 바이러스 감염증(코로나19) 장기화로 개인 위생수칙 준수가 더욱 중요...
3,세계일보,풀무원식품 신임 대표에 김진홍 전 LG전자 글로벌마케팅센터장 선임,"\n김 신임 대표 ""한국을 대표하는 바른 먹거리 기업인 풀무원식품이 글로벌 1위의 ..."
4,세계일보,"풀무원, 60대 이상 고연령층 대상으로 '바른먹거리 교육' 진행했더니…","\n시니어 바른먹거리 교육, 대사증후군 32.6% 개선 효과\n\n\n\n풀무원(대..."
5,세계일보,"풀무원, 온라인 모니터 요원 ‘e-fresh’ 24기 모집",\n풀무원식품(대표 김진홍)은 바른먹거리를 사랑하는 풀무원 온라인 소비자 모니터 ‘...
6,세계일보,"풀무원다논, '내시피족' 위한 액티비아 요거트 활용 레시피 선보여",\n요거트 전문 기업 풀무원다논(대표 정희련)이 액티비아 공식 유튜브 채널을 통해 ...
7,세계일보,풀무원식품 신임 대표에 김진홍,\n\n\n\n풀무원은 풀무원식품의 신임 대표이사에 김진홍(55·사진) 전 LG전자...
8,세계일보,"풀무원 '바른먹거리 교육', 8년 만에 누적 어린이 10만명 돌파",\n국내 기업에서 처음으로 2010년 어린이들의 식습관 개선을 위해 시작한 풀무원의...
9,세계일보,"풀무원, 국내 첫 ‘기능성 표시 일반식품’ 1호·2호 등록",\n\n\n\n풀무원이 일반식품에 대한 기능성 표시제 시행 후 국내 첫 기능성 표시...


In [12]:
biz.to_csv('세계일보.csv',encoding='utf-8-sig',index=None)